# Implement functionality for directed graphs

In [1]:
# API 2: we augment the AdjacencySpectralEmbed class with
# a predict method. This was Hayden's idea in the
# issue

import numpy as np
from scipy.linalg import orthogonal_procrustes
from sklearn.utils.validation import check_is_fitted

from sklearn.metrics.pairwise import euclidean_distances
from tqdm.notebook import tqdm

from graspy.simulations import sbm
from graspy.embed.base import BaseEmbed
from graspy.plot import heatmap
from graspy.utils import (
    import_graph,
    is_fully_connected,
    augment_diagonal,
    pass_to_ranks,
    is_unweighted,
)

import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
mpl.rcParams['figure.dpi'] = 120

## Define class

In [68]:
class AdjacencySpectralEmbed(BaseEmbed):
    r"""
    bla bla bla
    """

    def __init__(
        self,
        n_components=None,
        n_elbows=2,
        algorithm="randomized",
        n_iter=5,
        check_lcc=True,
        diag_aug=True,
    ):
        super().__init__(
            n_components=n_components,
            n_elbows=n_elbows,
            algorithm=algorithm,
            n_iter=n_iter,
            check_lcc=check_lcc,
        )

        if not isinstance(diag_aug, bool):
            raise TypeError("`diag_aug` must be of type bool")
        self.diag_aug = diag_aug

    def fit(self, graph, y=None):
        # would add the pseudoinverse
        # as a class attribute in fit
        """
        bla bla bla
        """
        A = import_graph(graph)

        if self.check_lcc:
            if not is_fully_connected(A):
                msg = (
                    "Input graph is not fully connected. Results may not"
                    + "be optimal. You can compute the largest connected component by"
                    + "using ``graspy.utils.get_lcc``."
                )
                warnings.warn(msg, UserWarning)

        if self.diag_aug:
            A = augment_diagonal(A)

        self._reduce_dim(A)
        
        # for out-of-sample
        inv_eigs = np.diag(1/self.singular_values_)
        self.pinv_left_ = self.latent_left_ @ inv_eigs
        if self.latent_right_ is not None:
            self.pinv_right_ = self.latent_right_ @ inv_eigs
        
        self.is_fitted_ = True
        return self
    
    def predict(self, y):
        """
        Fit ASE model to input graph

        Parameters
        ----------
        y : array_like or tuple
            out-of-sample matrix.
            If tuple, graph is directed and y[0] contains edges from y to other nodes.

        Returns
        -------
        array_like or tuple
            Out-of-sample prediction.
        """
        # TODO: force passing tuple for directed graphs
            
        check_is_fitted(self, 'is_fitted_')
        if self.latent_right_ is None:  # undirected
            if not isinstance(y, np.ndarray):
                raise TypeError("Undirected graphs require array input")
            return y @ self.pinv_left_
        else:  # directed
            if not isinstance(y, tuple):
                raise TypeError("Directed graphs require a tuple (y_right, y_left) as input")
            return y[0] @ self.pinv_left_, y[1] @ self.pinv_right_

## Test on directed graph

In [70]:
p = np.array([[.9, .1,],
              [.1, .9]])

M = sbm(n=[50, 50], p=p, directed=True)

# embed original matrix
idx = 0
a1 = M[idx, 1:]  # vector containing edges from a to other nodes
a2 = M[1:, idx]  # vector containing edges from other nodes to a
A = np.delete(np.delete(M, idx, 0), idx, 1)

ase = AdjacencySpectralEmbed(n_components=2)
ase.fit(A)


ase.predict((a1, a2))

(array([0.63560049, 0.64849359]), array([0.63563454, 0.63747695]))